In [2]:
import pandas as pd
import numpy as np
import datetime
import time
from calendar import monthrange
from pprint import pprint

In [3]:
df = pd.read_csv("../../../Data/vw_Incident.csv",encoding='latin-1',low_memory=False)

In [4]:
from sklearn.utils import resample
df = resample(df, n_samples=10, random_state=1234)

In [60]:
def days_left_in_quarter(date):
    q2 = (datetime.datetime.strptime("4/1/{0:4d}".format(date.year), "%m/%d/%Y")).timetuple().tm_yday
    q3 = (datetime.datetime.strptime("7/1/{0:4d}".format(date.year), "%m/%d/%Y")).timetuple().tm_yday
    q4 = (datetime.datetime.strptime("10/1/{0:4d}".format(date.year), "%m/%d/%Y")).timetuple().tm_yday
    q5 = (datetime.datetime.strptime("12/31/{0:4d}".format(date.year), "%m/%d/%Y")).timetuple().tm_yday + 1 # 1st Jan
    cur_day =  date.timetuple().tm_yday
    if (date.month < 4):
        return q2 - (cur_day)
    elif (date.month < 7):
#         return q3 - cur_day
        return q3 - (cur_day - q2 + 1)
    elif (date.month < 10):
        return q4 - (cur_day - q3 + 1)
    else:
        return q5 - (cur_day - q4 + 1)
def days_left_in_month(date):
    return int(monthrange(date.year, date.month)[1]) - int(date.strftime("%d"))

def get_last_bdays_months():
    last_bdays = pd.date_range("2017.01.01", periods=11, freq='BM')
    last_bdays_offset = []
    for last_bday in last_bdays:
        last_bdays_offset.append(last_bday + pd.DateOffset(days=1,hours=8))
    return last_bdays_offset

def get_last_bdays_qtrs():
    start_date = pd.datetime(2017, 1, 1) 
    last_bdays = pd.date_range(start_date, periods=3, freq='Q')
    last_bdays_offset = []
    for last_bday in last_bdays:
        last_bdays_offset.append(last_bday + pd.DateOffset(days=1,hours=8))
    last_bdays_offset = [start_date] + last_bdays_offset
    return last_bdays_offset

def get_seconds_left(date, last_bdays_offset):
    for i in range(len(last_bdays_offset)):       
        if date >= last_bdays_offset[i] and date <last_bdays_offset[i+1]:
            seconds_left = (last_bdays_offset[i+1] - date).seconds
            days_left = (last_bdays_offset[i+1] - date).days
            total_seconds_left = seconds_left + days_left*24*60*60        
            return total_seconds_left#/60/60/24

def time_taken_seconds(df):  # replace start & finish with one new column, "TimeTaken"
    df["Created_On"] = pd.to_datetime(df["Created_On"])
    df["ResolvedDate"] = pd.to_datetime(df["ResolvedDate"])
    df["TimeTaken"] = (df["ResolvedDate"] - df["Created_On"]).astype('timedelta64[s]')  
    mean_time = sum(df["TimeTaken"].tolist()) / len(df["TimeTaken"])  # Calculate mean of time taken
    std_time = np.std(df["TimeTaken"].tolist())  # Calculate standard deviation of time taken
    df = df[df["TimeTaken"] < (mean_time + 4*std_time)]  # Remove outliers that are > 3 std from mean
    last_bdays = get_last_bdays_months()
    df["Seconds_left_month"] = df["Created_On"].apply(lambda x: int(get_seconds_left(x, last_bdays)))  # Day of the month
    last_bdays = get_last_bdays_qtrs()
    df["Seconds_left_Qtr"] = df["Created_On"].apply(lambda x: int(get_seconds_left(x, last_bdays)))  # Day of the Qtr
    df["Days_left_Month"] = df["Created_On"].apply(lambda x: int(days_left_in_month(x)))  # Day of the month    
    df["Days_left_QTR"] = df["Created_On"].apply(lambda x: int(days_left_in_quarter(x)))  # Day of the Qtr
    return df

def time_taken(df):  # replace start & finish with one new column, "TimeTaken"
    df["Created_On"] = pd.to_datetime(df["Created_On"])
    df["ResolvedDate"] = pd.to_datetime(df["ResolvedDate"])
    df["TimeTaken"] = (df["ResolvedDate"] - df["Created_On"]).astype('timedelta64[s]')  
    mean_time = sum(df["TimeTaken"].tolist()) / len(df["TimeTaken"])  # Calculate mean of time taken
    std_time = np.std(df["TimeTaken"].tolist())  # Calculate standard deviation of time taken
    df = df[df["TimeTaken"] < (mean_time + 4*std_time)]  # Remove outliers that are > 3 std from mean
    last_bdays_months = get_last_bdays_months()
    df["Seconds_left_month"] = df["Created_On"].apply(lambda x: int(get_seconds_left(x, last_bdays_months)))  # seconds left to
    last_bdays_qtr = get_last_bdays_qtr()
    df["Seconds_left_QTR"] = df["Created_On"].apply(lambda x: int(get_seconds_left(x, last_bdays_qtr)))  # Day of the Qtr
    return df

df = time_taken_seconds(df)
df[["Created_On", "Seconds_left_month", "Days_left_Month", "Seconds_left_Qtr", "Days_left_QTR"]]

,Created_On,Seconds_left_month,Days_left_Month,Seconds_left_Qtr,Days_left_QTR
27439,2017-05-31 20:23:31,41789,0,2633789,121
34086,2017-03-07 12:47:25,2142755,24,2142755,25
23924,2017-04-10 18:49:42,1602618,20,7045818,172
17048,2017-04-20 01:58:28,799292,10,6242492,162
32399,2017-03-07 10:33:45,2150775,24,2150775,25
9449,2017-04-13 23:42:46,1325834,17,6769034,169
23706,2017-04-05 13:47:37,2052743,25,7495943,177
8222,2017-03-28 06:58:26,349294,3,349294,4
32427,2017-03-07 17:18:58,2126462,24,2126462,25
33950,2017-03-06 00:55:59,2271841,25,2271841,26


In [56]:
df = time_taken(df)
df[["Created_On", "Seconds_left_month", "Days_left_Month", "Seconds_left_Qtr", "Days_left_QTR"]]

,Created_On,Seconds_left_month,Days_left_Month,Seconds_left_Qtr,Days_left_QTR
27439,2017-05-31 20:23:31,0,0,30,121
34086,2017-03-07 12:47:25,24,24,24,25
23924,2017-04-10 18:49:42,18,20,81,172
17048,2017-04-20 01:58:28,9,10,72,162
32399,2017-03-07 10:33:45,24,24,24,25
9449,2017-04-13 23:42:46,15,17,78,169
23706,2017-04-05 13:47:37,23,25,86,177
8222,2017-03-28 06:58:26,4,3,4,4
32427,2017-03-07 17:18:58,24,24,24,25
33950,2017-03-06 00:55:59,26,25,26,26


In [59]:
date = pd.datetime(2017,5,31, 20,23,31)
print(date)
# date = pd.datetime(2017, 4,10, 18,49,42)
cutoff = pd.datetime(2017, 7, 1, 8,0,0)
print(cutoff)
print(cutoff - date)
print(date + pd.DateOffset(days=121))
print(pd.datetime(2017,4,20,1,58,28)+pd.DateOffset(days=121))

2017-05-31 20:23:31
2017-07-01 08:00:00
30 days, 11:36:29
2017-09-29 20:23:31
2017-08-19 01:58:28


In [78]:
def days_left_in_quarter(date):
    # Function found on stack overflow
    # https://stackoverflow.com/questions/37471704/how-do-i-get-the-correspondent-day-of-the-quarter-from-a-date-field
    q2 = (datetime.datetime.strptime("4/1/{0:4d}".format(date.year), "%m/%d/%Y")).timetuple().tm_yday
    q3 = (datetime.datetime.strptime("7/1/{0:4d}".format(date.year), "%m/%d/%Y")).timetuple().tm_yday
    q4 = (datetime.datetime.strptime("10/1/{0:4d}".format(date.year), "%m/%d/%Y")).timetuple().tm_yday
    q5 = (datetime.datetime.strptime("12/31/{0:4d}".format(date.year), "%m/%d/%Y")).timetuple().tm_yday + 1 # 1st Jan

    cur_day =  date.timetuple().tm_yday
    if (date.month < 4):
        return q2 - (cur_day - 1 + 1)
    elif (date.month < 7):
        return q3 - (cur_day - q2 + 1)
    elif (date.month < 10):
        return q4 - (cur_day - q3 + 1)
    else:
        return q5 - (cur_day - q4 + 1)
    
date = pd.datetime(2017,5,31, 20,23,31)
print(date)
print(days_left_in_quarter(date))

cur_day =  date.timetuple().tm_yday
q2 = (datetime.datetime.strptime("4/1/{0:4d}".format(date.year), "%m/%d/%Y")).timetuple().tm_yday
q3 = (datetime.datetime.strptime("7/1/{0:4d}".format(date.year), "%m/%d/%Y")).timetuple().tm_yday
print("cur_day=",cur_day, "q2=", q2,"q3=", q3)

print(q3 - (cur_day - q2 + 1))
print(182 - (151 - 91 + 1))
print("day in q2 = ", 151 - 91 + 1)
# q3 - day in q2 = 


2017-05-31 20:23:31
121
cur_day= 151 q2= 91 q3= 182
121
121
day in q2 =  61


In [ ]:
# print((start + pd.offsets.BMonthEnd(1)).day)
# # df['BMonthEnd'] = (df.index + pd.offsets.BMonthEnd(1)).day


# last_bdayslist = []
# for end_of_month in end_of_months:
#     last_bdayslist += pd.bdate_range(end_of_month- BDay(3),end_of_month- BDay(0))

# dflast_bdays = pd.DataFrame(last_bdayslist, columns=["Last_4_BDays"])
# dflast_bdays["Last_4_BDays"]
# # first_created_on = df["Created_On"].min()
# # last_created_on  = df["Created_On"].max()
# last_bdays = pd.date_range("2017.01.01", periods=11, freq='BM')
# last_bdays_offset = []
# for last_bday in last_bdays:
#     last_bdays_offset.append(last_bday + pd.DateOffset(days=1,hours=8))
# print(last_bdays_offset)

# def seconds_left_in_month(date, last_bdays_offset):
#     print(date.month)
#     cutoff = (date + pd.offsets.BMonthEnd(0)).normalize()
#     print(date)
#     print(cutoff)
    

# date = pd.datetime(2017, 4, 30, 7)
# seconds_left_in_month(date,last_bdays_offset)

In [350]:
# custom month end
def custom_month_end(date):
    
    wkday, days_in_month = monthrange(date.year, date.month)
    lastBDay = days_in_month - max(((wkday + days_in_month - 1) % 7) - 4, 0)
    m=0
    d=31
    increment_day = False
    if date.day == days_in_month and date.day == lastBDay:
#         print("lastBDay is last day in month")
        m=1
        d=1
        increment_day = False
    elif date.day > lastBDay + 1:
#         print("date is a sunday two days after the last business day")
        m=1
        d=31
        increment_day = True
    elif date.day > lastBDay and date.hour >= 8:
#         print("date day after the last business day and outside cutoff")
        m=1
        d=31
        increment_day = True
    elif date.day == 1 and date.hour < 8:
#         print("date is the first day of the month and within cutoff")
        d=1
        increment_day = False
#     elif date.day == 1 and date.hour >= 8:
#         print("date is the first day of the month and outside cutoff")
#         m=1
#         d=31
    elif lastBDay == days_in_month:
#         print("date is during the month and the last business day is the last day of the month")
        increment_day = False
        m=1
        d=1
        
#     elif lastBDay < days_in_month:
        
#     else: print("else")
    date += pd.DateOffset(months=m, day=d)
    
    if date.weekday() > 4:
#         print("weekend, therefore go back to the last business day and add 1 day")
        date-=pd.offsets.BDay()
        date+=pd.offsets.Day()

    elif increment_day == True:
#         print("increment day")
        date+=pd.offsets.Day()
    
        
#     print(date.normalize() + pd.DateOffset(hours=8))
    return date.normalize() + pd.DateOffset(hours=8)
dates = [pd.datetime(2017, 4, 28, 7), pd.datetime(2017, 4, 29, 7), pd.datetime(2017, 4, 30, 12), pd.datetime(2017, 6, 1, 7),
         pd.datetime(2017, 6, 1, 8), pd.datetime(2017, 7, 5, 8), pd.datetime(2017, 7, 31, 8), pd.datetime(2017, 8, 1, 7),
         pd.datetime(2017, 8, 1, 8), pd.datetime(2017, 8, 5, 8), pd.datetime(2017, 8, 20, 16), pd.datetime(2017, 8, 31, 16),
         pd.datetime(2017, 9, 1, 3), pd.datetime(2017, 9, 1, 12), pd.datetime(2017, 9, 10, 12), pd.datetime(2017, 11, 6, 12),
         pd.datetime(2017, 11, 6, 12), pd.datetime(2017, 12, 1, 2), pd.datetime(2017, 12, 1, 9),pd.datetime(2017, 9, 30, 7),
         pd.datetime(2017, 9, 30, 8)
        ]
should_bes = [pd.datetime(2017, 4, 29, 8), pd.datetime(2017, 4, 29, 8), pd.datetime(2017, 6, 1, 8), pd.datetime(2017, 6, 1, 8),
         pd.datetime(2017, 7, 1, 8), pd.datetime(2017, 8, 1, 8), pd.datetime(2017, 8, 1, 8), pd.datetime(2017, 8, 1, 8),
         pd.datetime(2017, 9, 1, 8), pd.datetime(2017, 9, 1, 8), pd.datetime(2017, 9, 1, 8), pd.datetime(2017, 9, 1, 8), 
         pd.datetime(2017, 9, 1, 8), pd.datetime(2017, 9, 30, 8), pd.datetime(2017, 9, 30, 8),pd.datetime(2017, 12, 1, 8),
         pd.datetime(2017, 12, 1, 8), pd.datetime(2017, 12, 1, 8), pd.datetime(2017, 12, 30, 8),pd.datetime(2017, 9, 30, 8),
         pd.datetime(2017, 11, 1, 8)
              
             ]
for date, should_be in zip(dates, should_bes):
    print()
    custom_month_end(date)
    print(should_be)
    
    print() 


2017-04-29 08:00:00


2017-04-29 08:00:00


2017-06-01 08:00:00


2017-06-01 08:00:00


2017-07-01 08:00:00


2017-08-01 08:00:00


2017-08-01 08:00:00


2017-08-01 08:00:00


2017-09-01 08:00:00


2017-09-01 08:00:00


2017-09-01 08:00:00


2017-09-01 08:00:00


2017-09-01 08:00:00


2017-09-30 08:00:00


2017-09-30 08:00:00


2017-12-01 08:00:00


2017-12-01 08:00:00


2017-12-01 08:00:00


2017-12-30 08:00:00


2017-09-30 08:00:00


2017-11-01 08:00:00



In [353]:
def seconds_left_in_month(date):    
    cutoff = custom_month_end(date)
#     print(date)
#     print(cutoff)
#     print(cutoff-date)
    seconds_left = (cutoff-date).seconds
    days_left = (cutoff-date).days
#     print(days_left, seconds_left)
    total_seconds_left = seconds_left + days_left*24*60*60
    return total_seconds_left

date = pd.datetime(2017, 4, 29, 7)
seconds_left_in_month(date)

3600

In [22]:
start = pd.datetime(2017, 2, 1)
end = pd.datetime(2017, 7, 1)
print((start + pd.offsets.BMonthEnd(1)).day)
# df['BMonthEnd'] = (df.index + pd.offsets.BMonthEnd(1)).day
end_of_months = pd.date_range(start, end, freq='BM')

last_bdayslist = []
for end_of_month in end_of_months:
    last_bdayslist += pd.bdate_range(end_of_month- BDay(3),end_of_month- BDay(0))

dflast_bdays = pd.DataFrame(last_bdayslist, columns=["Last_4_BDays"])
dflast_bdays["Last_4_BDays"]

28


NameError: name 'BDay' is not defined